In [1]:
# Importing packages
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import IFrame
import matplotlib.patheffects as PathEffectså

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching of roads
import gr_placematch # Contains functions that perform the map matching of places
import gr_utils # Contains useful geometry functions
import gr_plot # Contains plotting routines
import gr_process

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [151]:
trailname = 'gr131' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)
n1 = 2300
n2 = 2400
sc = 1.20 # scale factor
buffersize = 0.00015 # how many degs to buffer by

# Settings for trail2roads
points_per_batch = 100 # Subdivide the trail into batches of this many points
delta = 0.005 # Tolerance around bounding box per trail section [deg]

# Settings for roads2places
points_per_batch_places = 100 # Subdivide the trail into batches of this many segments
delta_places = 0.015 # bbox delta in deg

# Settings for places2processed
tracktype_p0 = ['grade4','grade5']
tracktype_p1 = ['grade2','grade3']
tracktype_p2 = ['grade1']
surface_p0   = ['ground','grass','dirt','sand','earth','mud']
surface_p1   = ['unpaved','gravel','fine_gravel','wood','compacted','rocks','pebblestone','woodchips','snow','ice','salt']
highway_p1   = ['track','path','footway','bridleway']
tol_d        = 0.5 # Consider a segment developed if it lies closer than tol_d to a developed area
types_slow   = ['pedestrian','track','footway','bridleway','steps','corridor','path']
types_heavy  = ['motorway','trunk','primary','secondary','tertiary']

In [69]:
trail = gr_utils.get_gpx(trailname)

Loading trail points from <data_input/gr131.gpx>...
Finished loading.


In [70]:
data_roads = gr_utils.read_roads(trailname)
bbox = gr_placematch.get_bbox(data_roads[n1:n2], delta_places);
places_landuse, places_admin8, places_admin9 = gr_placematch.get_places(bbox) # Grab relevant place information
# places_landuse = gr_placematch.get_all_places(bbox) # Grab relevant place information

In [71]:
i_focus = int(np.round((n1+n2)/2))
focus = [data_roads.iloc[i_focus]['x0'],data_roads.iloc[i_focus]['y0']]

## Drawing all polygons

In [72]:
# Map setup
trail_coords  = gr_mapmatch.trail_to_coords(trail)
chart = folium.Map(location=focus, zoom_start=13, tiles="OpenStreetMap")

# Draw development areas
for i in range(places_landuse.shape[0]):

    item = places_landuse.iloc[i]
    x,y = item['geometry'].exterior.coords.xy
    coords = pd.DataFrame({'LAT':y,'LON':x})
    newline = folium.Polygon(locations=coords, color='blue', fill=True, fillOpacity=0.5, weight=1)
    newline.add_to(chart)

# Draw trail
coords = gr_plot.get_fullcoords_from_frame(data_roads)
newline = folium.PolyLine(locations=coords[n1:n2], weight=2, color='red')
newline.add_to(chart)

# Draw bounding box
x,y = bbox.exterior.coords.xy
coords = pd.DataFrame({'LAT':y,'LON':x})
newline = folium.Polygon(locations=coords, color='black', fill=False, weight=2)
newline.add_to(chart)

# Render the map
filepath = f"cache/plot_all_devs.html"
chart.save(filepath)

In [73]:
IFrame(filepath, width=1000, height=500)

## Drawing buffered polygons

In [128]:
# Map setup
chart = folium.Map(location=focus, zoom_start=13, tiles="OpenStreetMap")

# Draw development areas
for i in range(places_landuse.shape[0]):

    item = places_landuse.iloc[i]
#     item_geom = shapely.affinity.scale(item['geometry'], xfact=sc, yfact=sc, origin='center')
    item_geom = item['geometry'].buffer(buffersize)
    x,y = item_geom.exterior.coords.xy
    coords = pd.DataFrame({'LAT':y,'LON':x})
    newline = folium.Polygon(locations=coords, color='blue', fill=True, fillOpacity=0.5, weight=1)
    newline.add_to(chart)

# Draw trail
coords = gr_plot.get_fullcoords_from_frame(data_roads)
newline = folium.PolyLine(locations=coords[n1:n2], weight=2, color='red')
newline.add_to(chart)

# Draw bounding box
x,y = bbox.exterior.coords.xy
coords = pd.DataFrame({'LAT':y,'LON':x})
newline = folium.Polygon(locations=coords, color='black', fill=False, weight=2)
newline.add_to(chart)

# Render the map
filepath = f"cache/plot_scaled_devs.html"
chart.save(filepath)

In [129]:
IFrame(filepath, width=1000, height=500)

## Drawing merged polygons

In [152]:
polys = []
for i in range(places_landuse.shape[0]):
    poly = places_landuse.iloc[i]['geometry'] # Grab polygon
#     poly = shapely.affinity.scale(poly, xfact=sc, yfact=sc, origin='center') # Scale it
    poly = poly.buffer(buffersize)
    polys.append(poly)
merged = list(shapely.ops.unary_union(polys))

/var/folders/zh/x06gqp657rl9h1shq_pn7gjh0000gn/T/ipykernel_6093/1521081679.py:7: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged = list(shapely.ops.unary_union(polys))
/var/folders/zh/x06gqp657rl9h1shq_pn7gjh0000gn/T/ipykernel_6093/1521081679.py:7: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged = list(shapely.ops.unary_union(polys))


In [153]:
# Map setup
chart = folium.Map(location=focus, zoom_start=13, tiles="OpenStreetMap")

# Draw development areas
k = 0
for poly in merged:

    x,y = poly.exterior.coords.xy
    coords = pd.DataFrame({'LAT':y,'LON':x})
    newline = folium.Polygon(locations=coords, color='blue', fill=True, fillOpacity=0.5, popup=k, weight=1)
    newline.add_to(chart)
    k += 1

# Draw trail
coords = gr_plot.get_fullcoords_from_frame(data_roads)
newline = folium.PolyLine(locations=coords[n1:n2], weight=2, color='red')
newline.add_to(chart)

# Draw bounding box
x,y = bbox.exterior.coords.xy
coords = pd.DataFrame({'LAT':y,'LON':x})
newline = folium.Polygon(locations=coords, color='black', fill=False, weight=2)
newline.add_to(chart)

# Render the map
filepath = f"cache/plot_merged_devs.html"
chart.save(filepath)

In [154]:
IFrame(filepath, width=1000, height=500)

## Drawing large merged polygons

In [161]:
tol_area = 15.0e-6

# Map setup
chart = folium.Map(location=focus, zoom_start=13, tiles="OpenStreetMap")

# Draw development areas
k = 0
for poly in merged:

    x,y = poly.exterior.coords.xy
    coords = pd.DataFrame({'LAT':y,'LON':x})
    if poly.area>tol_area:
        newline = folium.Polygon(locations=coords, color='blue', fill=True, fillOpacity=0.5, popup=k, weight=2)
    else:
        newline = folium.Polygon(locations=coords, color='green', fill=True, fillOpacity=0.1, popup=k, weight=1)
    newline.add_to(chart)
    k += 1

# Draw trail
coords = gr_plot.get_fullcoords_from_frame(data_roads)
newline = folium.PolyLine(locations=coords[n1:n2], weight=2, color='red')
newline.add_to(chart)

# Draw bounding box
x,y = bbox.exterior.coords.xy
coords = pd.DataFrame({'LAT':y,'LON':x})
newline = folium.Polygon(locations=coords, color='black', fill=False, weight=2)
newline.add_to(chart)

# Render the map
filepath = f"cache/plot_large_devs.html"
chart.save(filepath)

In [162]:
IFrame(filepath, width=1000, height=500)